In [5]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense
from keras import backend as K

In [29]:
K.set_image_dim_ordering('tf')

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(150, 150, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(2))
model.add(Activation('softmax'))

from keras.optimizers import Adam

model.compile(optimizer=Adam(lr=0.001), 
              loss='categorical_crossentropy', 
              metrics=['accuracy'])

In [6]:
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        'dataset/train',  # this is the target directory
        target_size=(150, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels

# this is a similar generator, for validation data
validation_generator = valid_datagen.flow_from_directory(
        'dataset/validation',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'dataset/test',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode=None,
        shuffle=False)

Found 22998 images belonging to 2 classes.
Found 2002 images belonging to 2 classes.
Found 0 images belonging to 0 classes.


In [7]:
from keras.preprocessing import image
import numpy as np

x_train = []
for i in train_generator.filenames:
    img = image.load_img('dataset/train/'+ i , target_size=(150,150, 3))
    img = image.img_to_array(img)
    img = img/255
    x_train.append(img)
x_train = np.array(x_train)

x_valid = []
for i in validation_generator.filenames:
    img = image.load_img('dataset/validation/'+ i , target_size=(150,150, 3))
    img = image.img_to_array(img)
    img = img/255
    x_valid.append(img)
x_valid = np.array(x_valid)

def create_labels(length): 
    a = []
    for i in range(length):
        if i < length/2:
            a.append(0)
        else:
            a.append(1)
    a = np.array(a)
    return a

y_train = create_labels(len(x_train))
y_valid = create_labels(len(x_valid))

from keras.utils import to_categorical

y_train = to_categorical(y_train)
y_valid = to_categorical(y_valid)

In [32]:
print('x_train shape:', x_train.shape)
print('Number of images in x_train', y_train.shape[0])
print('Number of images in x_valid', y_valid.shape[0])

x_train shape: (22998, 150, 150, 3)
Number of images in x_train 22998
Number of images in x_valid 2002


In [33]:
model.fit(x=x_train,y=y_train, epochs=25, batch_size=128)

Epoch 1/25
22998/22998 [==============================] - 277s 12ms/step - loss: 0.6579 - acc: 0.6012
Epoch 2/25
22998/22998 [==============================] - 275s 12ms/step - loss: 0.5335 - acc: 0.7373
Epoch 3/25
22998/22998 [==============================] - 259s 11ms/step - loss: 0.4651 - acc: 0.7846
Epoch 4/25
22998/22998 [==============================] - 244s 11ms/step - loss: 0.4231 - acc: 0.8101
Epoch 5/25
22998/22998 [==============================] - 242s 11ms/step - loss: 0.3834 - acc: 0.8296
Epoch 6/25
22998/22998 [==============================] - 241s 10ms/step - loss: 0.3570 - acc: 0.8436
Epoch 7/25
22998/22998 [==============================] - 240s 10ms/step - loss: 0.3276 - acc: 0.8599
Epoch 8/25
22998/22998 [==============================] - 240s 10ms/step - loss: 0.2918 - acc: 0.8744
Epoch 9/25
22998/22998 [==============================] - 241s 10ms/step - loss: 0.2714 - acc: 0.8844
Epoch 10/25
22998/22998 [==============================] - 240s 10ms/step - loss: 

In [34]:
model.evaluate(x_valid, y_valid)
model.save('cat_dog.h5')

2002/2002 [==============================] - 7s 4ms/step


In [8]:
from os.path import dirname, abspath, join
import sys

sys.path.append('/home/muskaanjain/git/deeplift')

import deeplift
from deeplift.layers import NonlinearMxtsMode
from deeplift.conversion import kerasapi_conversion as kc

revealcancel_model = kc.convert_model_from_saved_files(
                            h5_file='cat_dog.h5',
                            nonlinear_mxts_mode=NonlinearMxtsMode.RevealCancel)

nonlinear_mxts_mode is set to: RevealCancel
150
148
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
74
72
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
36
34
Heads-up: current implementation assumes maxpool layer is followed by a linear transformation (conv/dense layer)
Heads-up: I assume softmax is the output layer, not an intermediate one; if it's an intermediate layer, please let me know and I will prioritise that use-case


In [9]:
from deeplift.util import compile_func
import numpy as np
from keras import backend as K
from keras.models import load_model

saved_model_file = 'cat_dog.h5'
model = load_model(saved_model_file)
deeplift_model = revealcancel_model
deeplift_prediction_func = compile_func([deeplift_model.get_layers()[0].get_activation_vars()],
                                       deeplift_model.get_layers()[-1].get_activation_vars())
original_model_predictions = model.predict(x_valid, batch_size=16)
converted_model_predictions = deeplift.util.run_function_in_batches(
                                input_data_list=[x_valid],
                                func=deeplift_prediction_func,
                                batch_size=16,
                                progress_update=None)
print("difference in predictions:",np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)))
assert np.max(np.array(converted_model_predictions)-np.array(original_model_predictions)) < 10**-5
predictions = converted_model_predictions

difference in predictions: 0.0


[array([0.9982803 , 0.00171977], dtype=float32),
 array([1.000000e+00, 8.586603e-16], dtype=float32),
 array([9.9999964e-01, 3.6097165e-07], dtype=float32),
 array([0.9705768 , 0.02942318], dtype=float32),
 array([1.0000000e+00, 1.9756404e-12], dtype=float32),
 array([0.9988938 , 0.00110621], dtype=float32),
 array([9.999567e-01, 4.328460e-05], dtype=float32),
 array([1.000000e+00, 2.382692e-11], dtype=float32),
 array([9.9993634e-01, 6.3709347e-05], dtype=float32),
 array([0.06944519, 0.9305548 ], dtype=float32),
 array([9.999374e-01, 6.254846e-05], dtype=float32),
 array([9.9997783e-01, 2.2152652e-05], dtype=float32),
 array([0.39374235, 0.6062577 ], dtype=float32),
 array([1.000000e+00, 2.730575e-16], dtype=float32),
 array([0.7943506 , 0.20564935], dtype=float32),
 array([9.9999988e-01, 1.6791039e-07], dtype=float32),
 array([1.0000000e+00, 1.2193513e-14], dtype=float32),
 array([0.08535742, 0.9146426 ], dtype=float32),
 array([0.974596, 0.025404], dtype=float32),
 array([1.0000000

array([[9.9828029e-01, 1.7197676e-03],
       [1.0000000e+00, 8.5866027e-16],
       [9.9999964e-01, 3.6097165e-07],
       ...,
       [6.0935519e-09, 1.0000000e+00],
       [2.5441095e-01, 7.4558902e-01],
       [2.8922684e-03, 9.9710780e-01]], dtype=float32)

2010008

NameError: name 'keras' is not defined